# Retrieval Augmented Generation using LangChain
-Load enviroment variables into the main program

In [21]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Load enviroment file
load_dotenv()

openai_client = OpenAI()
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [22]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data/book_abt_chairs.pdf"

if not os.path.exists(file_path):
    print("The PDF file does not exist!!")
    sys.exit(1)

loader = PyPDFLoader(file_path)
pages = loader.load()

In [23]:
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = OpenAIEmbeddings()
vector_store = InMemoryVectorStore.from_documents(pages, embedding=embeddings)

retriever = vector_store.as_retriever()

In [24]:
def format_doc(docs):
    return "\n\n".join([doc.page_content for doc in docs if doc.page_content])

In [25]:
def rag(question: str):
    docs = retriever(question)
    context = format_doc(docs)
    system_message = (
        "Отговори на въпроса САМО на базата на тази информация:\n\n"
        f"{context}\n\n"
        "Ако отговорът не е намерен, кажи 'не мога да отговоря на базата на информацията от документа.'"
    )

    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ]
    )
    return response.choices[0].message.content.strip()

In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Отговори на въпроса САМО на базата на тази информация:\n\n{context}\n\n"
               "Ако отговорът не е намерен, кажи 'не мога да отговоря на базата на информацията от документа.'"),
    ("user", "{question}")
])

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_doc, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
question = "Има ли тема за динамичното седене?"
response = rag_chain.invoke(question)
print("Отговор:", response)

Отговор: Да, в документа е разгледана темата за динамичното седене, като се акцентира на неговите предимства и механизми, които осигуряват чести смени на позите и активиране на мускулите, което води до по-здравословен начин на живот.
